In [2]:
### Steps
# Preprocess data in R 
# Load in data
# Create training and testing data
# Run model (figure out cv search and how to alter this)
# Run model with random cv search
# Figure out how to evaluate the model (get multiple measures at once)
# Save the model
# Export the predicted values
# Load in the model and run again with new data from FHHC
pd.options.display.max_columns = None
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

In [129]:
house_dat = pd.read_csv("house_dat_8_13_20.csv", delimiter = ",")

In [130]:
house_dat.head()

,Quarter,Gender,RaceWhite,RaceBlack,Agegroup,OverallHealth,CapableManagingHealthCareNeeds,HandlingDailyLife,ControlLife,DealWithCrisis,GetsAlongWithFamily,SocialSituations,FunctioningHousing,Symptoms,Nervous,Hopeless,Restless,Depressed,EverythingEffort,Worthless,PsychologicalEmotionalProblems,LifeQuality,EnoughEnergyForEverydayLife,PerformDailyActivitiesSatisfaction,HealthSatisfaction,RelationshipSatisfaction,SelfSatisfaction,Tobacco_Use,Alcohol_Use,Cannabis_Use,ViolenceTrauma,Housing,Education,Employment,EnoughMoneyForNeeds,Friendships,EnjoyPeople,BelongInCommunity,SupportFromFamily,SupportiveFamilyFriends,GenerallyAccomplishGoal,telehealth,Housing.y,grant,EverServed,ActiveDuty_Else,NightsHomeless,aiety,mdd_r,mdd_s,another_s_ident,drug_use,er_hos_use_base,jail_arrest_base
0,2,1,0,1,8.0,4.0,3.0,2.0,3.0,2.0,3.0,2.0,4.0,2.0,4.0,2.0,0.0,0.0,0.0,0.0,4.0,4.0,2.0,2.0,2.0,2.4,4.0,4.0,1.0,2.0,1.0,1,12.0,0,4.0,4.0,2.0,4.0,3.0,3.0,4.0,0,1,0,0,1,0.0,0,0,0,0,9.0,0.0,0.0
1,1,0,1,0,6.0,1.0,1.0,4.0,4.0,4.0,4.0,4.0,3.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,4.0,5.0,4.0,5.0,3.0,4.0,2.0,1.0,1.0,1.0,0,13.0,1,2.0,5.0,4.0,4.0,4.0,4.0,4.0,0,1,0,0,0,12.0,0,0,0,1,12.0,0.0,0.0
2,1,0,1,0,8.0,2.0,1.0,2.0,4.0,4.0,1.0,5.0,1.0,2.0,4.0,4.0,4.0,3.0,3.0,4.0,5.0,2.0,4.0,5.0,5.0,3.2,1.6,4.0,1.0,1.0,1.0,0,14.0,0,3.0,2.0,4.0,4.0,2.0,5.0,5.0,0,1,0,0,0,26.0,0,0,0,0,9.0,4.0,2.0
3,1,1,1,0,7.0,4.0,2.0,2.0,2.0,4.0,1.0,4.0,1.0,2.0,4.0,3.0,2.0,2.0,4.0,2.0,4.0,1.0,1.0,2.0,2.0,2.0,3.0,1.0,2.0,2.0,1.0,0,14.0,0,1.0,3.0,2.0,1.0,2.0,2.0,4.0,0,0,0,0,1,30.0,0,1,0,0,9.0,0.0,0.0
4,2,0,0,1,7.0,2.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,5.0,5.0,4.0,4.0,4.0,4.0,1.0,1.0,1.0,1.0,1,14.0,0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,0,1,0,0,1,0.0,0,0,0,0,12.0,0.0,0.0


In [86]:
house_dat.shape

(2101, 54)

In [131]:
### Break into testing and training
from sklearn.model_selection import train_test_split
y = house_dat[["Housing.y"]]
predictors = house_dat.drop(["Housing.y"], axis = 1)
### Need to make this a horitzontal array not sure why
y = y.values.ravel()
predictors = predictors.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(predictors, y, test_size = .20, random_state = 53)

In [88]:
y_train

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [116]:
### Figure out how to get set up a Random search for xgboost (should be the same as previous random search)
from sklearn.model_selection import RandomizedSearchCV
xclas = XGBClassifier()  # and for classifier  
xclas.fit(X_train, y_train)  
pred_xclas =  xclas.predict(X_test)
pred_xclas

array([1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,

In [114]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
scoring = ['accuracy', 'precision', 'balanced_accuracy']
cv_results = cross_validate(xclas.fit(X_train, y_train) , X_train, y_train, cv=5, scoring=scoring)
cv_results

{'fit_time': array([0.15203905, 0.14604545, 0.14196086, 0.15003562, 0.1440444 ]),
 'score_time': array([0.0030129 , 0.00198865, 0.00399923, 0.00200105, 0.00299549]),
 'test_accuracy': array([0.86011905, 0.86607143, 0.85416667, 0.86011905, 0.83333333]),
 'test_precision': array([0.8650519 , 0.88970588, 0.86363636, 0.88321168, 0.86545455]),
 'test_balanced_accuracy': array([0.73449612, 0.7809437 , 0.7337093 , 0.76828548, 0.7288578 ])}

In [145]:
### Get a confusion matrix and then get sensitivity by hand
con_mat = confusion_matrix(y_test, pred_xclas)
#sensitivity <- TP / (TP + FN)
#specificity <- TN / (TN + FP)
con_mat
#tn, fp, fn, tp
sensitivity =  con_mat[1][1] / (con_mat[1][1] + con_mat[1][0])
specificity = con_mat[0][0] / (con_mat[0][0] + con_mat[0][1])
### 
balanced_accuracy =  (sensitivity+specificity)/2


In [231]:
### Fix this later
sensitivity = pd.DataFrame(sensitivity, columns = ["sensitivity"], index = [0])
specificity = pd.DataFrame(specificity, columns = ["specificity"], index = [0])
balanced_accuracy = pd.DataFrame(balanced_accuracy, columns = ["balanced_accuracy"], index = [0])

frames = [sensitivity, specificity, balanced_accuracy]
metrics_dat = pd.concat(frames, axis = 1)
metrics_dat

,sensitivity,specificity,balanced_accuracy
0,0.931373,0.591304,0.761338


In [174]:
# Figure out good values for these
## sum(negative instances) / sum(positive instances)
## Good for unbalanced weights
#Description of the parameters: https://xgboost.readthedocs.io/en/latest/parameter.html
y_panda = pd.DataFrame(y)
y_panda.describe()

,0
count,2101.000000
mean,0.757734
std,0.428556
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [190]:
# Develop develop one for random grid search
params = {  
    "max_depth": [6,12,15, 18],
    "eta": [.01, .001, .3],
    "colsample_bytree": [.5, .6, .7],
    "subsample": [.5, .6, .7],
    'lambda': [1, 1.1, 1.2, 1.3, 1.4],
    'alpha': [0, .1, .2, .3],
    "scale_pos_weight ": [.25, .5, 1],
}

In [191]:
xclas_random = XGBClassifier()
gs = RandomizedSearchCV(xclas_random, params, n_jobs=-1, cv = 10)  
gs.fit(X_train, y_train)  

RandomizedSearchCV(cv=10, estimator=XGBClassifier(), n_jobs=-1,
                   param_distributions={'alpha': [0, 0.1, 0.2, 0.3],
                                        'colsample_bytree': [0.5, 0.6, 0.7],
                                        'eta': [0.01, 0.001, 0.3],
                                        'lambda': [1, 1.1, 1.2, 1.3, 1.4],
                                        'max_depth': [6, 12, 15, 18],
                                        'scale_pos_weight ': [0.25, 0.5, 1],
                                        'subsample': [0.5, 0.6, 0.7]})

In [213]:
#best_estimator_
#best_score_ 
#best_params_
gs.best_estimator_
gs.best_score_

0.8720238095238095

In [207]:
pred_best_random =  gs.predict(X_test)
pred_best_random

array([1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,

In [208]:
### Get a confusion matrix and then get sensitivity by hand
con_mat = confusion_matrix(y_test, pred_best_random)
#sensitivity <- TP / (TP + FN)
#specificity <- TN / (TN + FP)
con_mat
#tn, fp, fn, tp
sensitivity =  con_mat[1][1] / (con_mat[1][1] + con_mat[1][0])
specificity = con_mat[0][0] / (con_mat[0][0] + con_mat[0][1])
### 
balanced_accuracy =  (sensitivity+specificity)/2

In [234]:
### Fix this later
##
accuracy = pd.DataFrame(gs.best_score_, columns = ["accuracy"], index = [0])
### Fix this later
sensitivity = pd.DataFrame(sensitivity, columns = ["sensitivity"], index = [0])
specificity = pd.DataFrame(specificity, columns = ["specificity"], index = [0])
balanced_accuracy = pd.DataFrame(balanced_accuracy, columns = ["balanced_accuracy"], index = [0])

frames = [sensitivity, specificity, balanced_accuracy, accuracy]
metrics_dat = pd.concat(frames, axis = 1)
metrics_dat = metrics_dat.round(2)
metrics_dat

,sensitivity,specificity,balanced_accuracy,accuracy
0,0.93,0.59,0.76,0.87


In [257]:
### Need to get the predicted probabilities
pred_prob =  gs.predict_proba(X_test)
prob_housed =   pd.DataFrame(pred_prob[:,1], columns = ["prob_housed"])
def if_else(row):

    if row['prob_housed'] < .25:

        val = "very high risk"

    elif row['prob_housed'] < .5:

        val = "high risk"
    
    elif row['prob_housed'] <.75:
        
        val = "medium risk"
    
    else:

        val = "low risk"

    return val

prob_housed['risk_level'] = prob_housed.apply(if_else, axis=1)
prob_housed
#prob_housed['risk_level'].value_counts()
        

,prob_housed,risk_level
0,0.996691,low risk
1,0.303953,high risk
2,0.961712,low risk
3,0.982547,low risk
4,0.175637,very high risk
...,...,...
416,0.090415,very high risk
417,0.651270,medium risk
418,0.950425,low risk
419,0.880083,low risk


In [90]:
### Now save the model
import pickle
import joblib
filename = "gs.sav"
load_model = joblib.load(filename)
load_model

RandomizedSearchCV(cv=10, estimator=XGBClassifier(missing=nan), n_jobs=-1,
                   param_distributions={'alpha': [0, 0.1, 0.2, 0.3],
                                        'colsample_bytree': [0.5, 0.6, 0.7],
                                        'eta': [0.01, 0.001, 0.3],
                                        'lambda': [1, 1.1, 1.2, 1.3, 1.4],
                                        'max_depth': [6, 12, 15, 18],
                                        'scale_pos_weight ': [0.25, 0.5, 1],
                                        'subsample': [0.5, 0.6, 0.7]})

In [266]:
new_dat_pred = house_dat.drop(["Housing.y"], axis = 1)
list(new_dat_pred.columns.values) 

['Quarter',
 'Gender',
 'RaceWhite',
 'RaceBlack',
 'Agegroup',
 'OverallHealth',
 'CapableManagingHealthCareNeeds',
 'HandlingDailyLife',
 'ControlLife',
 'DealWithCrisis',
 'GetsAlongWithFamily',
 'SocialSituations',
 'FunctioningHousing',
 'Symptoms',
 'Nervous',
 'Hopeless',
 'Restless',
 'Depressed',
 'EverythingEffort',
 'Worthless',
 'PsychologicalEmotionalProblems',
 'LifeQuality',
 'EnoughEnergyForEverydayLife',
 'PerformDailyActivitiesSatisfaction',
 'HealthSatisfaction',
 'RelationshipSatisfaction',
 'SelfSatisfaction',
 'Tobacco_Use',
 'Alcohol_Use',
 'Cannabis_Use',
 'ViolenceTrauma',
 'Housing',
 'Education',
 'Employment',
 'EnoughMoneyForNeeds',
 'Friendships',
 'EnjoyPeople',
 'BelongInCommunity',
 'SupportFromFamily',
 'SupportiveFamilyFriends',
 'GenerallyAccomplishGoal',
 'telehealth',
 'grant',
 'EverServed',
 'ActiveDuty_Else',
 'NightsHomeless',
 'aiety',
 'mdd_r',
 'mdd_s',
 'another_s_ident',
 'drug_use',
 'er_hos_use_base',
 'jail_arrest_base']

In [80]:
### Replicate data cleaning from r ehr_machine_learning
fhhc_house_test_dat = pd.read_csv("fhhc_house_test_dat.csv", delimiter = ",")
### Get two specific ids for baseline then subset
#'1096', '1095'
### Create filter variable
new_clients = ["1096'", "'1095'"]
fhhc_house_test_dat = fhhc_house_test_dat[fhhc_house_test_dat["ConsumerID"].isin(new_clients)]
### Now get the variables from above
predictors = fhhc_house_test_dat[['Quarter',
 'Gender',
 'RaceWhite',
 'RaceBlack',
 'Agegroup',
 'OverallHealth',
 'CapableManagingHealthCareNeeds',
 'HandlingDailyLife',
 'ControlLife',
 'DealWithCrisis',
 'GetsAlongWithFamily',
 'SocialSituations',
 'FunctioningHousing',
 'Symptoms',
 'Nervous',
 'Hopeless',
 'Restless',
 'Depressed',
 'EverythingEffort',
 'Worthless',
 'PsychologicalEmotionalProblems',
 'LifeQuality',
 'EnoughEnergyForEverydayLife',
 'PerformDailyActivitiesSatisfaction',
 'HealthSatisfaction',
 'RelationshipSatisfaction',
 'SelfSatisfaction',
 'Tobacco_Use',
 'Alcohol_Use',
 'Cannabis_Use',
 'ViolenceTrauma',
 'Housing',
 'Education',
 'Employment',
 'EnoughMoneyForNeeds',
 'Friendships',
 'EnjoyPeople',
 'BelongInCommunity',
 'SupportFromFamily',
 'SupportiveFamilyFriends',
 'GenerallyAccomplishGoal',
 'EverServed',
 'ActiveDuty_Else',
 'NightsHomeless']]
predictors

,Quarter,Gender,RaceWhite,RaceBlack,Agegroup,OverallHealth,CapableManagingHealthCareNeeds,HandlingDailyLife,ControlLife,DealWithCrisis,GetsAlongWithFamily,SocialSituations,FunctioningHousing,Symptoms,Nervous,Hopeless,Restless,Depressed,EverythingEffort,Worthless,PsychologicalEmotionalProblems,LifeQuality,EnoughEnergyForEverydayLife,PerformDailyActivitiesSatisfaction,HealthSatisfaction,RelationshipSatisfaction,SelfSatisfaction,Tobacco_Use,Alcohol_Use,Cannabis_Use,ViolenceTrauma,Housing,Education,Employment,EnoughMoneyForNeeds,Friendships,EnjoyPeople,BelongInCommunity,SupportFromFamily,SupportiveFamilyFriends,GenerallyAccomplishGoal,EverServed,ActiveDuty_Else,NightsHomeless
219,4,1,0,1,8,1,3,2,3,3,4,4,4,1,2,1,0,3,0,1,5,4,5,4,4,3,2,4,1,3,1,4,11,1,3,4,4,4,4,4,3,0,0,0
221,4,1,0,1,5,1,1,5,5,2,5,5,1,2,4,4,4,4,4,4,5,1,1,1,1,1,1,1,1,1,1,3,12,3,2,1,4,1,3,2,5,0,1,30


In [81]:
### Variables to create
#'telehealth', 'grant','aiety','mdd_r', 'mdd_s', 'another_s_ident', 'drug_use', 'er_hos_use_base', 'jail_arrest_base'
telehealth = [1,1]
telehealth = pd.DataFrame(data = telehealth, columns = ["telehealth"], index = [219, 221])
### Remember CCBHC or not
grant = [0, 0]
## Index 219 221
grant = pd.DataFrame(data = grant, columns = ["grant"], index = [219, 221])

def if_else(row):

    if row['DiagnosisOne'] == 62:
        val= 1
    else:

        val = 0

    return val

aiety =  fhhc_house_test_dat.apply(if_else, axis=1)
aiety = pd.DataFrame(data = aiety, columns = ["aiety"])


def if_else(row):

    if row['DiagnosisOne'] == 59:
        val= 1
    else:

        val = 0

    return val

mdd_r =  fhhc_house_test_dat.apply(if_else, axis=1)
mdd_r = pd.DataFrame(data = mdd_r, columns = ["mdd_r"])

def if_else(row):

    if row['DiagnosisOne'] == 58:
        val= 1
    else:

        val = 0

    return val

mdd_s =  fhhc_house_test_dat.apply(if_else, axis=1)
mdd_s = pd.DataFrame(data = mdd_s, columns = ["mdd_s"])
mdd_s


def if_else(row):

    if row['SexualIdentity'] > 1:
        val= 1
    else:

        val = 0

    return val

another_s_ident =  fhhc_house_test_dat.apply(if_else, axis=1)
another_s_ident = pd.DataFrame(data = another_s_ident, columns = ["another_s_ident"])

def if_else(row):

    if row['Housing'] == 1:
        val= 1
    else:

        val = 0

    return val

Housing =  fhhc_house_test_dat.apply(if_else, axis=1)
Housing = pd.DataFrame(data = Housing, columns = ["Housing"])
Housing

drug_use = fhhc_house_test_dat["Cocaine_Use"] +fhhc_house_test_dat["Meth_Use"] + fhhc_house_test_dat["StreetOpioids_Use"] +  fhhc_house_test_dat["RxOpioids_Use"] + fhhc_house_test_dat["Stimulants_Use"]  + fhhc_house_test_dat["Inhalants_Use"] +  fhhc_house_test_dat["Sedatives_Use"] + fhhc_house_test_dat["Hallucinogens_Use"] + fhhc_house_test_dat["Other_Use"]
drug_use = pd.DataFrame(data = drug_use, columns = ["drug_use"])
drug_use

er_hos_use_base = fhhc_house_test_dat["NightsDetox"] + fhhc_house_test_dat["NightsHospitalMHC"] + fhhc_house_test_dat["TimesER"]
er_hos_use_base = pd.DataFrame(data = er_hos_use_base,columns = ["er_hos_use_base"])
er_hos_use_base

jail_arrest_base = fhhc_house_test_dat["NightsJail"] + fhhc_house_test_dat["NumTimesArrested"]
jail_arrest_base = pd.DataFrame(data = jail_arrest_base, columns = ["jail_arrest_base"])
jail_arrest_base

#### Need reformat these variables and drop from predictors
### Change gender to male or female / another gender identity
#machine_dat$Gender.x = ifelse(machine_dat$Gender.x == 1,1,0)

#### Employment 
#machine_dat$Employment.x = ifelse(machine_dat$Employment.x <3,1,0)

def if_else(row):

    if row['Gender'] > 1:
        val= 1
    else:

        val = 0

    return val

Gender =  fhhc_house_test_dat.apply(if_else, axis=1)
Gender = pd.DataFrame(data = Gender, columns = ["Gender"])

def if_else(row):

    if row['Employment'] < 3:
        val= 1
    else:

        val = 0

    return val

Employment =  fhhc_house_test_dat.apply(if_else, axis=1)
Employment = pd.DataFrame(data = Employment, columns = ["Employment"])
### Variables to drop Gender, Employment

In [82]:
predictors = predictors.drop(["Gender", "Employment"], axis = 1)
predictors

,Quarter,RaceWhite,RaceBlack,Agegroup,OverallHealth,CapableManagingHealthCareNeeds,HandlingDailyLife,ControlLife,DealWithCrisis,GetsAlongWithFamily,SocialSituations,FunctioningHousing,Symptoms,Nervous,Hopeless,Restless,Depressed,EverythingEffort,Worthless,PsychologicalEmotionalProblems,LifeQuality,EnoughEnergyForEverydayLife,PerformDailyActivitiesSatisfaction,HealthSatisfaction,RelationshipSatisfaction,SelfSatisfaction,Tobacco_Use,Alcohol_Use,Cannabis_Use,ViolenceTrauma,Housing,Education,EnoughMoneyForNeeds,Friendships,EnjoyPeople,BelongInCommunity,SupportFromFamily,SupportiveFamilyFriends,GenerallyAccomplishGoal,EverServed,ActiveDuty_Else,NightsHomeless
219,4,0,1,8,1,3,2,3,3,4,4,4,1,2,1,0,3,0,1,5,4,5,4,4,3,2,4,1,3,1,4,11,3,4,4,4,4,4,3,0,0,0
221,4,0,1,5,1,1,5,5,2,5,5,1,2,4,4,4,4,4,4,5,1,1,1,1,1,1,1,1,1,1,3,12,2,1,4,1,3,2,5,0,1,30


In [83]:
#### Get quarter and gender
Quarter = predictors.iloc[:,0]
Quarter = pd.DataFrame(Quarter)
## Put together gender and quarter
#frames = [Quarter, Gender]
#quarter_gender = pd.concat(frames, axis = 1)
#quarter_gender
Education = predictors.iloc[:,31]
Education  = pd.DataFrame(Education)
Education

,Education
219,11
221,12


In [84]:
### Now get race RaceWhite to ViolenceTrauma
RaceWhite_to_ViolenceTrauma =  predictors.iloc[:,1:30]
#Need 'EnoughMoneyForNeeds' to 'NightsHomeless',
EnoughMoneyForNeeds_to_GenerallyAccomplishGoal =  predictors.iloc[:,32:39]
EnoughMoneyForNeeds_to_GenerallyAccomplishGoal
EverServed_to_NightsHomeless = predictors.iloc[:,39:42]
EverServed_to_NightsHomeless

,EverServed,ActiveDuty_Else,NightsHomeless
219,0,0,0
221,0,1,30


In [61]:
EverServed_to_NightsHomeless

,EverServed,ActiveDuty_Else,NightsHomeless
219,0,0,0
221,0,1,30


In [85]:
### Add variables to predictors
frames_post =  [aiety, mdd_r, mdd_s, another_s_ident,drug_use , er_hos_use_base , jail_arrest_base]
post_pred = pd.concat(frames_post, axis = 1)
post_pred

,aiety,mdd_r,mdd_s,another_s_ident,drug_use,er_hos_use_base,jail_arrest_base
219,0,0,1,0,10,0,0
221,1,0,0,0,9,0,0


In [137]:
### Reorder predictors and put into numpy array

frames = [Quarter, Gender, RaceWhite_to_ViolenceTrauma, Housing, Education, Employment, EnoughMoneyForNeeds_to_GenerallyAccomplishGoal, telehealth, grant, EverServed_to_NightsHomeless, post_pred]
predictors_ordered = pd.concat(frames, axis = 1)
predictors_ordered.shape
### Need to rename as 'f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52'
predictors_ordered.rename(columns={"Quarter": "f0", "Gender": "f1", "RaceWhite": "f2", "RaceBlack": "f3", "Agegroup": "f4", "OverallHealth": "f5", "CapableManagingHealthCareNeeds": "f6", "HandlingDailyLife": "f7", "ControlLife": "f8", "DealWithCrisis": "f9", "GetsAlongWithFamily": "f10", "SocialSituations": "f11", "FunctioningHousing": "f12", "Symptoms": "f13","Nervous": "f14","Hopeless": "f15", "Restless": "f16", "Depressed": "f17", "EverythingEffort": "f18", "Worthless": "f19", "PsychologicalEmotionalProblems": "f20", "LifeQuality": "f21", "EnoughEnergyForEverydayLife": "f22", "PerformDailyActivitiesSatisfaction": "f23", "HealthSatisfaction": "f24", "RelationshipSatisfaction": "f25", "SelfSatisfaction": "f26", "Tobacco_Use": "f27", "Alcohol_Use": "f28", "Cannabis_Use": "f29", "ViolenceTrauma": "f30", "Housing": "f31", "Education": "f32", "Employment": "f33", "EnoughMoneyForNeeds": "f34", "Friendships": "f35", "EnjoyPeople": "f36", "BelongInCommunity": "f37", "SupportFromFamily": "f38", "SupportiveFamilyFriends" : "f39", "GenerallyAccomplishGoal": "f40", "telehealth": "f41", "grant": "f42", "EverServed": "f43", "ActiveDuty_Else": "f44", "NightsHomeless": "f45", "aiety": "f46", "mdd_r": "f47", "mdd_s": "f48", "another_s_ident": "f49", "drug_use": "f50", "er_hos_use_base": "f51", "jail_arrest_base": "f52"}, inplace = True)
predictors_ordered

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52
219,4,0,0,1,8,1,3,2,3,3,4,4,4,1,2,1,0,3,0,1,5,4,5,4,4,3,2,4,1,3,1,0,11,1,3,4,4,4,4,4,3,1,0,0,0,0,0,0,1,0,10,0,0
221,4,0,0,1,5,1,1,5,5,2,5,5,1,2,4,4,4,4,4,4,5,1,1,1,1,1,1,1,1,1,1,0,12,0,2,1,4,1,3,2,5,1,0,0,1,30,1,0,0,0,9,0,0


In [135]:
X_test.shape

(421, 53)

In [138]:
load_model
predictors_ordered = predictors_ordered.to_numpy()

### Check out the order variabes are in for X_test
pred_prob =  load_model.predict_proba(predictors_ordered)
#pred_prob =  load_model.predict_proba(X_test)


In [139]:
pred_prob

array([[0.43602866, 0.56397134],
       [0.32870328, 0.6712967 ]], dtype=float32)

In [140]:
prob_housed =   pd.DataFrame(pred_prob[:,1], columns = ["prob_housed"])
def if_else(row):

    if row['prob_housed'] < .25:

        val = "very high risk"

    elif row['prob_housed'] < .5:

        val = "high risk"
    
    elif row['prob_housed'] <.75:
        
        val = "medium risk"
    
    else:

        val = "low risk"

    return val

prob_housed['risk_level'] = prob_housed.apply(if_else, axis=1)
prob_housed


,prob_housed,risk_level
0,0.563971,medium risk
1,0.671297,medium risk
